<a href="https://colab.research.google.com/github/jeffblackadar/sensation/blob/main/sensory_map_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Package installation, Mount Google Drive

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install packages
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install geopandas
!pip install contextily

# 2. Relocate map coordinates to get ready to unroll map at your location's latitude longitude
* Clicking a location in Google Maps will provide this.

In [18]:
# Input the location to unroll the map at.
local_lat = 45.475871
local_lon = -75.549315

#local_lat = 45.477661
#local_lon = -75.536143

local_lat = 43.586368
local_lon = -79.542910

local_lat = 43.590619
local_lon = -79.539579

In [19]:
shapefiles_path = "/content/drive/MyDrive/github/sensation/shapefiles/"
from geographiclib.geodesic import Geodesic
import geopy
from geopy.distance import VincentyDistance
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os

def transition_point(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon):
    #print(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon)  
    geod = Geodesic.WGS84.Inverse(origin_lat, origin_lon, dest_lat, dest_lon)
    distance = geod['s12']
    bearing = geod['azi1']
    origin = geopy.Point(new_map_origin_lat, new_map_origin_lon)
    destination = VincentyDistance(meters=distance).destination(origin, bearing)
    lat2, lon2 = destination.latitude, destination.longitude
    return(lat2, lon2)


geotif_crs = 4326
new_map_shp = gpd.GeoDataFrame()
new_map_shp['geometry'] = None
new_map_shp.crs = ("EPSG:" + str(geotif_crs))
new_map_shp.geometry = new_map_shp.geometry.to_crs(crs=geotif_crs)
new_map_shp.to_crs(crs=geotif_crs)
new_map_shp = new_map_shp.to_crs(epsg=geotif_crs)

print("#--------------------------------------------------------")
print("# Copy this below, paste in the cell in section 3 of this notebook to provide instructions to generate the new map")
print("#--------------------------------------------------------")

h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))
for index, row in h_df.iterrows():
    #print(row['type'], row['name'], row['geometry'])
    p = row['geometry']
    lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p.y, p.x, local_lat, local_lon)

    point = Point(lon2, lat2)
    new_p_row = {'id':row['id'], 'geometry':point, 'type':row['type'], 'name':row['name']}
    t_name = str(row['name'])
    if(t_name.lower()=='none'):
        t_name=""
    t_desc = str(row['desc'])
    if(t_desc.lower()=='none'):
        t_desc=""
    print('gm.genMarker('+str(lat2)+', '+str(lon2)+',"'+str(row['type'])+'","'+t_name+'","'+t_desc+'")')
    new_map_shp = new_map_shp.append(new_p_row, ignore_index=True)

new_map_shp.to_file(os.path.join(shapefiles_path,"new_map.shp"))


new_map_shp_poly = gpd.GeoDataFrame()
new_map_shp_poly['geometry'] = None
new_map_shp_poly.crs = ("EPSG:" + str(geotif_crs))
new_map_shp_poly.geometry = new_map_shp_poly.geometry.to_crs(crs=geotif_crs)
new_map_shp_poly.to_crs(crs=geotif_crs)
new_map_shp_poly = new_map_shp_poly.to_crs(epsg=geotif_crs)
polygon_points_array = []

h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements_polys.shp"))
for index, row in h_df.iterrows():
    poly = row['geometry']
    polygon_points = ""
    comma = ""
    for p in poly.exterior.coords:
        lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p[1], p[0], local_lat, local_lon)
        polygon_points = polygon_points + comma + "["+str(lat2)+","+str(lon2)+"]"
        comma = ","
        #print(lat2, lon2)
    #/content/drive/MyDrive/crane_obs/crane_hopewell/hopewell_selected_elements.shp
        point = Point(lon2, lat2)
        polygon_points_array.append(point)
    polygon = Polygon(polygon_points_array)
    new_p_row = {'id':row['id'], 'geometry':polygon, 'type':row['type'], 'name':row['name']}
    print("gm.genPolygon('["+polygon_points+"]','"+str(row['type'])+"')")
    new_map_shp_poly = new_map_shp_poly.append(new_p_row, ignore_index=True)

new_map_shp_poly.to_file(os.path.join(shapefiles_path,"new_map_polys.shp"))

#test marker
origin = geopy.Point(local_lat, local_lon)
destination = VincentyDistance(meters=2000).destination(origin, 270)
lat2, lon2 = destination.latitude, destination.longitude
print('gm.genTestMarker('+str(lat2)+', '+str(lon2)+')')
print("")
print("#--------------------------------------------------------")
print("# Copy this above, paste in the cell in section 3 of this notebook to provide instructions to generate the new map")
print("#--------------------------------------------------------")

#--------------------------------------------------------
# Copy this below, paste in the cell in section 3 of this notebook to provide instructions to generate the new map
#--------------------------------------------------------
gm.genMarker(43.593654326758255, -79.55584456932061,"RCH","RCH","")
gm.genMarker(43.59279454398659, -79.55226532313628,"RCH","","")
gm.genMarker(43.593833719809794, -79.55053000083149,"RCH","","")
gm.genMarker(43.59455092539228, -79.54693483243337,"RCH","","")
gm.genMarker(43.59370195624827, -79.54549226155021,"RCH","","")
gm.genMarker(43.59413298261094, -79.54367569307601,"RCH","","")
gm.genMarker(43.594529847821654, -79.54272831745246,"RCH","","")
gm.genMarker(43.585036569602245, -79.54245862954197,"RCH","","")
gm.genMarker(43.585081673406165, -79.54078796613217,"RCH","","")
gm.genMarker(43.58449022888838, -79.54170792727547,"RCH","","")
gm.genMarker(43.5834781395323, -79.5421403073463,"RCH","","")
gm.genMarker(43.584029452677115, -79.54319130382235,"RCH","

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
import pandas as pd
import contextily as ctx

print("Original Map")
h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))

#print(h_df.crs)
# Change crs to one compatible with basemap
h_df = h_df.to_crs(epsg=3857) #3857

ax = h_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=15)

h_df['coords'] = h_df['geometry'].apply(lambda x: x.representative_point().coords[:])
h_df['coords'] = [coords[0] for coords in h_df['coords']]
for idx, row in h_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

print("Unrolled map at your location")
new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map.shp"))

print(new_map_df.crs)
# Change crs to one compatible with basemap
new_map_df = new_map_df.to_crs(epsg=3857) #3857
print(new_map_df.crs)

ax = new_map_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=13)

new_map_df['coords'] = new_map_df['geometry'].apply(lambda x: x.representative_point().coords[:])
new_map_df['coords'] = [coords[0] for coords in new_map_df['coords']]
for idx, row in new_map_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map_polys.shp"))
#new_map_df.boundary.plot(ax=ax,color="red",alpha=0.3)

# 3. Generate the map to display at your location

In [25]:
import json
class GenerateLeafletMap:
    def __init__(self,html_file_name):
        self.html_file_name = html_file_name
        self.html_file = open(self.html_file_name, 'w') 
        self.markers = 0
        self.polygons = 0
        self.polylines = 0
        self.layers = 0
        self.type_data = {}
    def close(self):
        self.html_file.close()

    def set_type_data_dict(self,type_data_dict):
        self.type_data = type_data_dict

    def genMapHTMLTop(self):
        self.html_file.write('<!DOCTYPE html>\n')
        self.html_file.write('<html>\n')
        self.html_file.write('<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n')
        self.html_file.write('<title>map title</title>\n')
        self.html_file.write('<link href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.css" rel="stylesheet" />\n')
        self.html_file.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.js"></script>\n')
        self.html_file.write('<script src="https://code.jquery.com/jquery-2.1.4.min.js"></script>\n')
        self.html_file.write('<style type="text/css">#my-map {\n')
        self.html_file.write('  width:960px;\n')
        self.html_file.write('  height:700px;\n')
        self.html_file.write('}\n')
        self.html_file.write('h1 {\n')
        self.html_file.write('  color: #AA0000;\n')
        self.html_file.write('  font-size: 24px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h2 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 20px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h3 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 16px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')    
        self.html_file.write('p {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('li {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')  
        self.html_file.write('th {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('td {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('</style>\n')
        self.html_file.write('</head>\n')
        self.html_file.write('<body>\n')

    def genMapHTMLH1(self,map_title):
        self.html_file.write('<h1>'+map_title+'</h1>\n')
        self.html_file.write('<div id="description">description</div>\n')
        self.html_file.write('<div id="latitude">latitude</div>\n')
        self.html_file.write('<div id="longitude">longitude</div>\n')
        self.html_file.write('<div id="description_point"></div>\n')
        self.html_file.write('<div id="description_poly"></div>\n')
                    
    def genMapHTMLScriptTop(self): 
        self.html_file.write('  <div id="my-map"></div>\n')
        self.html_file.write('    <script>\n')
        self.html_file.write('    // This file was generated by this R program:  https://github.com/jeffblackadar/oc-transpo-maps/blob/master/oc_4_gen_html.r\n')
        self.html_file.write('var map;\n')
        self.html_file.write("map = L.map('my-map');\n")
        self.html_file.write('var markers = [];\n')
        self.html_file.write('var polygons = [];\n')
        self.html_file.write('var markers_data = [];\n')
        self.html_file.write('var polygons_data = [];\n')

        self.html_file.write("// Thank you:\n")
        self.html_file.write("// https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula\n")
        self.html_file.write("function distance(lat1, lon1, lat2, lon2) {\n")
        self.html_file.write("  var p = 0.017453292519943295;    // Math.PI / 180\n")  
        self.html_file.write("  var c = Math.cos;\n")
        self.html_file.write("  var a = 0.5 - c((lat2 - lat1) * p)/2 + \n")
        self.html_file.write("          c(lat1 * p) * c(lat2 * p) * \n")
        self.html_file.write("          (1 - c((lon2 - lon1) * p))/2;\n")
        self.html_file.write("  return 12742 * Math.asin(Math.sqrt(a)); // 2 * R; R = 6371 km\n")
        self.html_file.write("}\n")

        self.html_file.write("//https://stackoverflow.com/questions/31790344/determine-if-a-point-reside-inside-a-leaflet-polygon\n")
        self.html_file.write("function isMarkerInsidePolygon(marker, poly) {\n")
        self.html_file.write("    var polyPoints = poly.getLatLngs();       \n")
        self.html_file.write("    var x = marker.getLatLng().lat, y = marker.getLatLng().lng;\n")
        self.html_file.write("    var inside = false;\n")
        self.html_file.write("    for (var i = 0, j = polyPoints.length - 1; i < polyPoints.length; j = i++) {\n")
        self.html_file.write("        var xi = polyPoints[i].lat, yi = polyPoints[i].lng;\n")
        self.html_file.write("        var xj = polyPoints[j].lat, yj = polyPoints[j].lng;\n")
        self.html_file.write("        var intersect = ((yi > y) != (yj > y)) && (x < (xj - xi) * (y - yi) / (yj - yi) + xi);\n")
        self.html_file.write("        if (intersect) inside = !inside;\n")
        self.html_file.write("    }\n")
        self.html_file.write("    return inside;\n")
        self.html_file.write("};\n")
        self.html_file.write("function update_per_proximity(marker) {\n")
        self.html_file.write("    var marker_lat = marker.getLatLng().lat, marker_lon = marker.getLatLng().lng;\n")

        self.html_file.write("// Polygons - check if marker is inside.  If so, light it up.\n")  
        self.html_file.write("    for (cn=0;cn<polygons.length;cn++){\n")
        self.html_file.write("        if(isMarkerInsidePolygon(marker, polygons[cn])==true){\n")
        self.html_file.write("            polygons[cn].setStyle({fillOpacity: 0.7, weight:2,opacity: 0.3});\n")
        self.html_file.write('            var popup_text_poly = polygons_data[cn].sight + " " + polygons_data[cn].hearing + " " + polygons_data[cn].feeling + " " + polygons_data[cn].smell;\n')        
        self.html_file.write("            document.getElementById('description_poly').innerHTML = popup_text_poly;\n")
        self.html_file.write("            var audio_temp = polygons_data[cn].sound;\n")
        self.html_file.write("            if(audio_temp.trim().length>0){audio_temp = '<audio controls><source src=\"sounds/'+audio_temp+'\"> Your browser does not support the audio element. </audio>'}")
        self.html_file.write("            polygons[cn]._popup.setContent(popup_text_poly + audio_temp)\n")

        self.html_file.write("        }\n")
        self.html_file.write("    }\n")
#    "hill":'{"color":"brown","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
        self.html_file.write("    for (cn=0;cn<markers.length;cn++){\n")
        self.html_file.write("        dist_to = distance(marker_lat, marker_lon,markers[cn].getLatLng().lat,markers[cn].getLatLng().lng)\n")
        self.html_file.write("        // adjust distance for proximity sensitivity\n")
        self.html_file.write("        if(dist_to < 0.1){\n")

        self.html_file.write("            if(markers_data[cn].type=='rch'){\n")
        self.html_file.write("                markers[cn].setIcon(iconRCH);\n")
        
        self.html_file.write("                //markers[cn]._popup.setContent('RCH  <audio controls><source src=\"sounds/rch.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(markers_data[cn].type=='furnace'){\n")
        self.html_file.write("                markers[cn].setIcon(iconFurnace);\n")
        self.html_file.write("                //markers[cn]._popup.setContent('Furnace  <audio controls><source src=\"sounds/furnace.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(markers_data[cn].type=='business'){\n")
        self.html_file.write("                markers[cn].setIcon(iconBusiness);\n")
        self.html_file.write("                //markers[cn]._popup.setContent('Business  <audio controls><source src=\"sounds/business.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")

        self.html_file.write("            if(markers_data[cn].type=='house'){\n")
        self.html_file.write("                markers[cn].setIcon(iconHouse);\n")
        self.html_file.write("                //markers[cn]._popup.setContent('House  <audio controls><source src=\"sounds/house.wav\" type=\"audio/wav\"> Your browser does not support the audio element. </audio>')\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")    
        self.html_file.write('            var desc_temp = markers_data[cn].desc;')
        self.html_file.write('            if(desc_temp.trim().length>0){desc_temp = "This is described on the map as: "+desc_temp+". "};')
        self.html_file.write('            var popup_text = markers_data[cn].sight + " " + markers_data[cn].hearing + " " + markers_data[cn].feeling + " " + markers_data[cn].smell + " " + desc_temp;\n')
        self.html_file.write("            document.getElementById('description_point').innerHTML = popup_text;\n")
        self.html_file.write("            var audio_temp = markers_data[cn].sound;\n")
        self.html_file.write("            if(audio_temp.trim().length>0){audio_temp = '<audio controls><source src=\"sounds/'+audio_temp+'\"> Your browser does not support the audio element. </audio>'}")
        self.html_file.write("            markers[cn]._popup.setContent(popup_text + audio_temp)\n")


        self.html_file.write("        } // if dist_to\n")
        self.html_file.write("    } // for\n")

        self.html_file.write("} // function\n")                      
      
        self.html_file.write('var iconHouse = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/house.png",\n')
        self.html_file.write('    //shadowUrl: "leaf-shadow.png",\n')

        self.html_file.write('    iconSize:     [30, 37], // size of the icon\n')
        self.html_file.write('    //shadowSize:   [50, 64], // size of the shadow\n')
        self.html_file.write('    iconAnchor:   [15, 36], // point of the icon which will correspond to markers location\n')
        self.html_file.write('    //shadowAnchor: [4, 62],  // the same for the shadow\n')
        self.html_file.write('    popupAnchor:  [-3, -38] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('var iconRCH = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/rch.png",\n')
        self.html_file.write('    iconSize:     [35, 33], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 32], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -34] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')
      
        self.html_file.write('var iconFurnace = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/furnace.png",\n')
        self.html_file.write('    iconSize:     [34, 38], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 37], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -39] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconBusiness = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/business.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconTravel = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/travel.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconComputer = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/computer.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconQuestion = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/question.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')


                    
    def genMapHTMLMapTop(self):
        self.html_file.write("\n")
                      
        self.html_file.write("      //basemap.fitBounds(geojson.getBounds());\n")
        self.html_file.write("      map.setView([45.468, -75.553], 15);\n")
        self.html_file.write("      basemap.addTo(map);\n")
        self.html_file.write("\n")
        self.html_file.write("      // add legend control layers - global variable with (null, null) allows indiv basemaps and overlays to be added inside functions below\n")
        self.html_file.write("      var controlLayers = L.control.layers(null, null, {\n")
        self.html_file.write('        position: "topleft",\n')
        self.html_file.write("        collapsed: false // false = open by default\n")
        self.html_file.write("      }).addTo(map);\n")
        self.html_file.write("\n")
        #writeLines("      var markers = [];", self$outputFileHtmlCon)
                      

    def genHTMLBaseMap(self):
        self.html_file.write('  window.onload = function() {\n')
        self.html_file.write("    var basemap = L.tileLayer('https://{s}.tile.osm.org/{z}/{x}/{y}.png', {\n")
        self.html_file.write("      attribution: '&copy; "+'<a href="https://osm.org/copyright">OpenStreetMap</a>'+"'\n")
        self.html_file.write("    });\n")

    def genMarker(self,marker_lat, marker_long, marker_type, marker_name, marker_desc):
        type_data_row = self.type_data[marker_type.lower()]
        #add more data to the json for the marker
        type_data_row = type_data_row[:-1]+',"name":"'+marker_name+'","desc":"'+marker_desc+'"}'
        type_data_row_json = json.loads(type_data_row)  
        self.html_file.write('  markers.push(L.marker(['+str(marker_lat)+','+str(marker_long)+'], {icon: iconQuestion}).addTo(map));\n')
        self.html_file.write('     markers['+str(self.markers)+"].bindPopup('Approach the landmark to see what it is.').openPopup();\n")
        self.html_file.write('     markers_data.push('+type_data_row+')\n')
        self.markers = self.markers +1


    def genPolygon(self,polygon_coods_json, polygon_type):
        # Thanks to: https://gis.stackexchange.com/questions/290609/entering-manual-coordinates-to-draw-polygon-in-leaflet
        fill_color="black"
        type_data_row = self.type_data[polygon_type.lower()]
        type_data_row_json = json.loads(type_data_row)

        fill_color = type_data_row_json['color']
        #if(polygon_type.lower()=="hill"):
        #    fill_color="DarkGreen"
        #if(tolower(polygon_type)=="forest"){fill_color="Green"}
        #if(tolower(polygon_type)=="field"){fill_color="Wheat"}
        #if(tolower(polygon_type)=="road"){fill_color="DarkGrey"}
        #if(tolower(polygon_type)=="creek"){fill_color="Blue"}
        
        self.html_file.write('var coords = '+polygon_coods_json+';\n')
                      #writeLines(paste0('var a = JSON.parse(coords);\n')
        self.html_file.write('polygons.push(L.polygon(coords, {\n')
        self.html_file.write('  color: "white",\n')
        self.html_file.write('  fillColor: "'+fill_color+'",\n')
        self.html_file.write('  weight: 0,\n')
        self.html_file.write('  opacity: 0,\n')
        self.html_file.write('  fillOpacity: 0\n')
        self.html_file.write('}).addTo(map));\n')
        self.html_file.write('     polygons['+str(self.polygons)+"].bindPopup('Approach the landmark to see what it is.').openPopup();\n")
        self.html_file.write('     polygons_data.push('+type_data_row+')\n')
        self.html_file.write('\n')
        self.polygons = self.polygons +1


    def genPolyline(self,p1_lat, p1_long, p2_lat, p2_long):
        self.polylines = self.polylines +1
        self.html_file.write('  var pointA = new L.LatLng(',paste0(p1_lat),',',paste0(p1_long),' );\n')
        self.html_file.write('  var pointB = new L.LatLng(',paste0(p2_lat),',',paste0(p2_long),');\n')
        self.html_file.write('  var pointList = [pointA, pointB];\n')
        self.html_file.write('  \n')
        self.html_file.write('  var polyline'+str(self.polylines)+' = new L.Polyline(pointList, {\n')
        self.html_file.write("      color: 'red',\n")
        self.html_file.write('      weight: 3,\n')
        self.html_file.write('      opacity: 0.5,\n')
        self.html_file.write('      smoothFactor: 1\n')
        self.html_file.write('  });\n')
        self.html_file.write('  polyline'+str(self.polylines)+'.addTo(map);\n')

    def genTestMarker(self,marker_lat, marker_long,):
        self.html_file.write("// To simulate walking around this is a draggable marker. This will greatly reduce testing time.\n")
        self.html_file.write("// Thanks to\n")
        self.html_file.write("// https://stackoverflow.com/questions/57384822/leaflet-detect-when-marker-goes-into-and-out-of-a-circle\n")
        self.html_file.write("var test_marker = new L.marker(["+str(marker_lat)+","+str(marker_long)+"],{\n")
 
        self.html_file.write("    draggable: true,\n")
        self.html_file.write("    autoPan: true\n")
        self.html_file.write("}).addTo(map);\n")
        self.html_file.write("test_marker.setIcon(iconComputer)\n")

        self.html_file.write("test_marker.on('dragend', function (e) {\n")
        self.html_file.write("    document.getElementById('latitude').innerHTML = test_marker.getLatLng().lat;\n")
        self.html_file.write("    document.getElementById('longitude').innerHTML = test_marker.getLatLng().lng;\n")
        self.html_file.write("    document.getElementById('description').innerHTML = distance(test_marker.getLatLng().lat,test_marker.getLatLng().lng,markers[13].getLatLng().lat,markers[13].getLatLng().lng);\n")

        self.html_file.write("    update_per_proximity(test_marker)\n")      
                    

    def genHTMLEndMap(self):
        self.html_file.write("      controlLayers.addOverlay(basemap, 'OpenStreetMap');\n")
        self.html_file.write("      \n")
        self.html_file.write("   // });\n")




        self.html_file.write("});\n")

        self.html_file.write("  };\n")


        self.html_file.write("   // https://gis.stackexchange.com/questions/182068/getting-current-user-location-automatically-every-x-seconds-to-put-on-leaflet\n")
        self.html_file.write("var current_position, current_accuracy;\n")
        self.html_file.write("function onLocationFound(e) {\n")
        self.html_file.write("    // if position defined, then remove the existing position marker and accuracy circle from the map\n")
        self.html_file.write("    if (current_position) {\n")
        self.html_file.write("        map.removeLayer(current_position);\n")
        #writeLines(paste0("       map.removeLayer(current_accuracy);\n")
        self.html_file.write("    }\n")
        self.html_file.write("    var radius = e.accuracy / 2;\n")
        self.html_file.write("    current_position = L.marker(e.latlng).addTo(map)\n")
        self.html_file.write('        .bindPopup("You are within " + radius + " meters from this point");\n')
        self.html_file.write("    current_position.setIcon(iconTravel)\n")
        #writeLines(paste0("current_accuracy = L.circle(e.latlng, radius).addTo(map);\n")
        self.html_file.write("    update_per_proximity(current_position);\n")
        self.html_file.write("}\n")
        self.html_file.write("function onLocationError(e) {\n")
        self.html_file.write("    alert(e.message);\n")
        self.html_file.write("}\n")
        self.html_file.write("map.on('locationfound', onLocationFound);\n")
        self.html_file.write("map.on('locationerror', onLocationError);\n")
        self.html_file.write("// wrap map.locate in a function    \n")
        self.html_file.write("function locate() {\n")
        self.html_file.write("    map.locate({setView: true, maxZoom: 16});\n")
        self.html_file.write("}\n")

        self.html_file.write("// call locate every 3 seconds... forever\n")
        self.html_file.write("setInterval(locate, 3000);\n")

        self.html_file.write("</script>\n")
                    
    def genHTMLEndBody(self):
        self.html_file.write("<hr>\n")
        #self.html_file.write("<p>Page last modifed: ", format(Sys.Date(),"%d %B, %Y"),"</p>\n")
        self.html_file.write("</body>\n")
        self.html_file.write("</html>\n")

    def genLayer(self,geojsonfile):
        self.layers = self.layers +1
        #print(paste0("To map for ", railYear, ", adding: "))
        #print(output_dbRows_route_style[i_route_style, 1])
        layerName = "layer" + str(self.layers)
                      
        self.html_file.write("\n")
        self.html_file.write("// layer for "+geojsonfile+"\n")
        self.html_file.write("var ",layerName,";\n")
                      
        self.html_file.write("      $.ajax({\n")
        self.html_file.write("        type: 'POST',\n")
        self.html_file.write("        url: '",geojsonfile,"',\n")

        self.html_file.write("        dataType: 'json',\n")
        self.html_file.write("        success: function(response) {\n")
        self.html_file.write("          ",layerName," = L.geoJson(response, {\n")
                      
        self.html_file.write("             onEachFeature: function (feature, layer) {\n")

        self.html_file.write("                 layer.bindPopup('<p>'+feature.properties.RTE_TYPE+'</p><p>'+feature.properties.MODE+'</p>');\n")  
                      
        self.html_file.write("               }\n")
        self.html_file.write("             });\n")
        self.html_file.write("          ",layerName,".setStyle({\n")
        self.html_file.write("            color: 'purple',\n")
        self.html_file.write("            weight: 2,\n")
        self.html_file.write('            opacity: 1\n')
        self.html_file.write("          });\n")
        self.html_file.write("          ",layerName,".addTo(map);\n")
        self.html_file.write("          controlLayers.addOverlay(",layerName,", '<span style=",'"color: purple"',">route</span>');\n")

        self.html_file.write("        }\n")
        self.html_file.write("      });\n")
        self.html_file.write("\n")
                      

gm = GenerateLeafletMap("/content/drive/MyDrive/github/sensation/userloc.html")
gm.genMapHTMLTop()


type_data_dict={
    "hill":'{"type":"hill","color":"brown","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp","sound":""}',
    "rch":'{"type":"rch","color":"","sight":"a flat circle","hearing":"wind blowing through trees","smell":"ash","feeling":"cool, damp","sound":"rch.wav"}',
    "forest":'{"type":"forest","color":"green","sight":"level terrain, trees","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp","sound":""}',
    "field":'{"type":"field","color":"wheat","sight":"level terrain, grass","hearing":"wind blowing through grass","smell":"grass","feeling":"warm, sun","sound":""}',
    "road":'{"type":"road","color":"gray","sight":"a muddy road","hearing":"squash of footsteps","smell":"mud","feeling":"sun","sound":"road.wav"}',
    "path":'{"type":"path","color":"gray","sight":"a muddy path","hearing":"squash of footsteps","smell":"mud","feeling":"sun","sound":"path.wav"}',
    "creek":'{"type":"creek","color":"blue","sight":"a running creek","hearing":"water rippling","smell":"fresh water","feeling":"sun","sound":"creek.wav"}',
    "business":'{"type":"business","color":"","sight":"a building","hearing":"horses","smell":"smoke","feeling":"cool, damp","sound":""}',
    "furnace":'{"type":"furnace","color":"","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp","sound":"furnace.wav"}',
    "house":'{"type":"house","color":"","sight":"a house","hearing":"people talking","smell":"wood smoke","feeling":"cool, damp","sound":""}'
}
gm.set_type_data_dict(type_data_dict)

gm.genMapHTMLH1("Hopewell, Pennsylvania")
gm.genMapHTMLScriptTop()
gm.genHTMLBaseMap()
gm.genMapHTMLMapTop()

#--------------------------------------------------------
# Paste below
#--------------------------------------------------------

gm.genMarker(43.593654326758255, -79.55584456932061,"RCH","RCH","")
gm.genMarker(43.59279454398659, -79.55226532313628,"RCH","","")
gm.genMarker(43.593833719809794, -79.55053000083149,"RCH","","")
gm.genMarker(43.59455092539228, -79.54693483243337,"RCH","","")
gm.genMarker(43.59370195624827, -79.54549226155021,"RCH","","")
gm.genMarker(43.59413298261094, -79.54367569307601,"RCH","","")
gm.genMarker(43.594529847821654, -79.54272831745246,"RCH","","")
gm.genMarker(43.585036569602245, -79.54245862954197,"RCH","","")
gm.genMarker(43.585081673406165, -79.54078796613217,"RCH","","")
gm.genMarker(43.58449022888838, -79.54170792727547,"RCH","","")
gm.genMarker(43.5834781395323, -79.5421403073463,"RCH","","")
gm.genMarker(43.584029452677115, -79.54319130382235,"RCH","","")
gm.genMarker(43.58217512941633, -79.54845725797645,"RCH","","")
gm.genMarker(43.59112425048376, -79.54517236557129,"Furnace","Clingan & Buckley's Furnace","Clingan & Buckley's Furnace")
gm.genMarker(43.590619000000004, -79.539579,"Business","Clingan & Buckley","Clingan & Buckley")
gm.genMarker(43.589264572969356, -79.53985528253607,"House","","")
gm.genMarker(43.58839414656474, -79.54004957076567,"House","","")
gm.genMarker(43.587635029542774, -79.53963940044787,"House","","")
gm.genMarker(43.58759027523924, -79.5384055100686,"House","","S.H. Is this a school house?")
gm.genMarker(43.58692829943205, -79.53946129600469,"House","Clingan & Buckley","Clingan & Buckley's house.")
gm.genMarker(43.58603183283075, -79.538087281139,"House","N. Care","N. Care's house.")
gm.genMarker(43.588328117417035, -79.54192130658078,"House","","")
gm.genMarker(43.58780120355106, -79.54380749004974,"House","","")
gm.genMarker(43.587233437003476, -79.54469333425592,"House","","")
gm.genPolygon('[[43.585370414219646,-79.53917641049433],[43.585345604836874,-79.53914380526521],[43.585345604836874,-79.53914380526521],[43.585345604836874,-79.53914380526521],[43.58529709403896,-79.53914533026933],[43.58529709403896,-79.53914533026933],[43.5849321550791,-79.53909003275164],[43.58100752221621,-79.54535581129879],[43.581175612313835,-79.55259458735569],[43.58250794891628,-79.55392172854934],[43.58283383620735,-79.55160809851162],[43.586141389786604,-79.54469417647033],[43.586757510835334,-79.54086887294721],[43.585370414219646,-79.53917641049433]]','hill')
gm.genPolygon('[[43.58509695640145,-79.53878438690735],[43.58516972258853,-79.53878209896286],[43.58514546719273,-79.53878286161165],[43.58514546719273,-79.53878286161165],[43.58514546719273,-79.53878286161165],[43.59025683013636,-79.53969054730713],[43.590302017859,-79.53948879946643],[43.58514325072861,-79.53864939121041],[43.58509695640145,-79.53878438690735]]','road')
gm.genPolygon('[[43.58876561716867,-79.53903628615464],[43.59000750164958,-79.53782866821457],[43.58989328947123,-79.53679724202142],[43.589831292309334,-79.53452882972209],[43.589301929761746,-79.53334356635834],[43.58867942547551,-79.5323949726938],[43.58528767480528,-79.53857807961795],[43.58876561716867,-79.53903628615464]]','field')
gm.genPolygon('[[43.5894578781047,-79.55891339277129],[43.58891384066319,-79.55829596156771],[43.58874581986095,-79.55693232699478],[43.588095417311116,-79.55575066305481],[43.587077323779106,-79.5543467860408],[43.58719647156274,-79.55274054084146],[43.58668550405835,-79.55118735511974],[43.58698210607486,-79.55004296895237],[43.58681233121671,-79.54857929661927],[43.587258851147794,-79.5476972911902],[43.58765023214374,-79.546416362052],[43.588591189706946,-79.54461738989816],[43.58852223487745,-79.54338423655071],[43.58876803399567,-79.54210781936487],[43.589061775930546,-79.54079649799587],[43.588813686242396,-79.54047041972915],[43.58859271443982,-79.5417794528271],[43.58838934831964,-79.54268728148435],[43.588188738018296,-79.543761950722],[43.587470509626684,-79.5458210370551],[43.58714969530225,-79.5469662132245],[43.586664593643,-79.54845037425753],[43.58660946090356,-79.54952044297856],[43.58644625139745,-79.55139514150403],[43.58690925486568,-79.55298320515621],[43.586885477449954,-79.5544863111526],[43.5878291556133,-79.55579234512541],[43.58848504388146,-79.5573077007941],[43.58869773380443,-79.55843623279803],[43.58919599946837,-79.55922202217772],[43.5894578781047,-79.55891339277129]]','creek')
gm.genPolygon('[[43.593287641490406,-79.5571581834228],[43.59712684194877,-79.54274699430529],[43.59510810830697,-79.54247643681714],[43.59171889970677,-79.54151436239017],[43.591425277337514,-79.54723301963335],[43.59207080144081,-79.5495834214298],[43.591215185015244,-79.55214768432923],[43.59180660018615,-79.55416592891338],[43.592781127132326,-79.55587177571665],[43.593287641490406,-79.5571581834228]]','forest')
gm.genPolygon('[[43.58932791620079,-79.56038094261672],[43.589570248670405,-79.55862464787143],[43.58929018902261,-79.55602188414859],[43.58915484425244,-79.55294829422388],[43.58953913463537,-79.55055798050496],[43.59037534650924,-79.54885301226778],[43.59125773599192,-79.54789271224585],[43.591133383652085,-79.54754684027397],[43.59025138209495,-79.54853044858935],[43.5893816809262,-79.55025978135717],[43.588863801941216,-79.55277084309127],[43.589095732160054,-79.55656422551776],[43.58924878209654,-79.55865796482638],[43.589116992540326,-79.5599211645253],[43.58932791620079,-79.56038094261672]]','path')
gm.genPolygon('[[43.583228863938686,-79.5574230356944],[43.582929088523706,-79.55568379823784],[43.5829341952605,-79.55393507010066],[43.58329384934569,-79.55210533065453],[43.58440497587157,-79.54959927030747],[43.58620208458324,-79.54655869803473],[43.58682496493743,-79.54528013888924],[43.58783507549462,-79.54282359733244],[43.588430539716214,-79.5409162769615],[43.58893176224266,-79.5404808335605],[43.58915344482341,-79.53954119693198],[43.58896595149341,-79.53947713993301],[43.58881009369635,-79.54029812249111],[43.58822610784429,-79.54085274842568],[43.586965462944825,-79.54455294339323],[43.584332986202064,-79.5493450573517],[43.58312138174789,-79.55192419954743],[43.58273015483968,-79.5538948045927],[43.5827619950271,-79.55582888905211],[43.5829913290257,-79.55740711477188],[43.583228863938686,-79.5574230356944]]','road')
gm.genPolygon('[[43.59357944211672,-79.55919947318807],[43.59276469015897,-79.55603022973946],[43.591671041717795,-79.55445537909972],[43.591142172625794,-79.55218675350264],[43.591891804254125,-79.54934189480835],[43.59124656699065,-79.54721685175907],[43.59154087790639,-79.54144816147435],[43.591354544837145,-79.5414540124262],[43.59109796426173,-79.54745467868479],[43.591702777303745,-79.54918458080053],[43.590936594492874,-79.55205326668151],[43.591572858236354,-79.55466829948557],[43.59259644584198,-79.55610542539492],[43.593395021571986,-79.55932180056337],[43.59357944211672,-79.55919947318807]]','road')
gm.genPolygon('[[43.59572110059016,-79.5421097500541],[43.59571569230888,-79.54178344839242],[43.59449760200584,-79.54191496775188],[43.59301769451259,-79.54158833399269],[43.59146732373324,-79.5411007054112],[43.59128169354604,-79.54012719430351],[43.590465897286805,-79.53998960732366],[43.59040695714473,-79.5395017961039],[43.590253342461835,-79.53943667292542],[43.59028304454986,-79.54020520869078],[43.591098453871496,-79.54031949279909],[43.59138765074838,-79.54140633752425],[43.59290220976971,-79.5417785063155],[43.594621199342555,-79.54221423391392],[43.59572110059016,-79.5421097500541]]','road')
gm.genPolygon('[[43.591229715379185,-79.54005887472229],[43.591159169309314,-79.53887189376746],[43.59114252609998,-79.53786976011075],[43.59097313257819,-79.53787508939642],[43.59106294994035,-79.53920136525188],[43.59109420047831,-79.54006313313188],[43.591229715379185,-79.54005887472229]]','road')
gm.genPolygon('[[43.59161923464449,-79.54925714629972],[43.5912789108416,-79.5481485432068],[43.59042808899009,-79.54896794904123],[43.58969274280711,-79.55062313184771],[43.58924030935051,-79.55299226085194],[43.5897196408383,-79.5584334652344],[43.593276062678825,-79.5593021816871],[43.59251366535675,-79.55622459437078],[43.59140538214225,-79.55479010878835],[43.5907702737678,-79.55224499821455],[43.59161923464449,-79.54925714629972]]','field')
gm.genPolygon('[[43.58914063673135,-79.55826494536169],[43.58876408565422,-79.55289053780339],[43.58922922629412,-79.55026454556061],[43.590195944335505,-79.5482523766939],[43.590995557557946,-79.54741124888324],[43.59126907516279,-79.54141006093548],[43.59098026496981,-79.54034652375587],[43.59030153054919,-79.54029789688063],[43.59015750820741,-79.53978944348165],[43.589206196864914,-79.53965612300104],[43.58904956451028,-79.5404304991935],[43.588673984118294,-79.54333355768367],[43.58866595765677,-79.54489602085566],[43.58750142100685,-79.54724081284397],[43.586999788329535,-79.54867878412682],[43.58702402766851,-79.55014692802902],[43.58682111748106,-79.55120248083743],[43.58732001117765,-79.5526791232361],[43.5871593994616,-79.55424630302515],[43.58881044452482,-79.5567363261357],[43.58898511837396,-79.55808325119555],[43.58914063673135,-79.55826494536169]]','field')
gm.genPolygon('[[43.588678582879524,-79.5590254496373],[43.58838189688826,-79.55747247911859],[43.587797985896884,-79.55597510387801],[43.58670433763953,-79.55440038681452],[43.5867667711972,-79.5530461252737],[43.58631677901875,-79.55145138869976],[43.5866178827411,-79.54813115036144],[43.588152723583605,-79.54358307187927],[43.58860109284233,-79.54098086996623],[43.588404004027026,-79.54136012397112],[43.587059031093354,-79.54508627726105],[43.58338085137091,-79.55224250151653],[43.583071245664506,-79.55402405351383],[43.58304728093101,-79.55565680168627],[43.58338246792161,-79.55748819793972],[43.588678582879524,-79.5590254496373]]','field')
gm.genPolygon('[[43.588731121388754,-79.53962442110215],[43.58560422368078,-79.53911649382196],[43.58687057293536,-79.54087200748373],[43.58624324161766,-79.54494863156927],[43.58636991546303,-79.54543429094863],[43.58817297048055,-79.54071451892241],[43.58869151820449,-79.5403018479671],[43.588731121388754,-79.53962442110215]]','field')
gm.genTestMarker(43.59061631750575, -79.56434507770508)
#--------------------------------------------------------
# Paste above
#--------------------------------------------------------

gm.genHTMLEndMap()
gm.genHTMLEndBody()

gm.close()

# 4. Test generated map using http server on Google Colab

To test the map, we can use an http server on Google Colab. 
* Run the cell below to generate the URL for the test map.
* Start the http server
* Click the URL to launch the map in a web browser

Once your map is ready, the userloc.html file can be placed on an http server that has TLS/https and accessed from a smartphone.

In [24]:
# Start the server, then click on the URL
from google.colab.output import eval_js
print("Click this URL when the server starts.")
print(eval_js("google.colab.kernel.proxyPort(8000)")+"drive/MyDrive/github/sensation/userloc.html")
!python -m http.server 8000

Click this URL when the server starts.
https://ugec3c354v-496ff2e9c6d22116-8000-colab.googleusercontent.com/drive/MyDrive/github/sensation/userloc.html
Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [25/Apr/2021 17:00:36] "GET /drive/MyDrive/github/sensation/userloc.html HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 17:00:36] "GET /drive/MyDrive/github/sensation/icons/question.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 17:00:36] "GET /drive/MyDrive/github/sensation/icons/computer.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 17:00:36] code 404, message File not found
127.0.0.1 - - [25/Apr/2021 17:00:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Apr/2021 17:00:48] "GET /drive/MyDrive/github/sensation/sounds/road.wav HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 17:00:50] code 404, message File not found
127.0.0.1 - - [25/Apr/2021 17:00:50] "GET /favicon.ico HTTP/1.1" 404 -

Keyboard interrupt received, exiting.
^C
